# Importing Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy import stats

# Importing Models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor

# Importing evaluation modules
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Setting environment to ignore future warnings
import warnings
warnings.simplefilter('ignore')
%matplotlib inline



# 1. Data Loading and Overview

## 1.1 Data Loading

In [ ]:
from google.colab import files
import io
from IPython.display import clear_output

message = None
while True:
    clear_output(wait=True)
    print("Please upload your sales CSV now")
    if message:
        print(message)
    uploaded = files.upload()

    if not uploaded:
        message = "No file uploaded. Please try again."
        continue

    fname = next(iter(uploaded))
    if not fname.lower().endswith('.csv'):
        message = f" {fname!r} is not a CSV. Please upload a .csv file."
        continue

    # Success
    df = pd.read_csv(io.BytesIO(uploaded[fname]), parse_dates=['InvoiceDate'])
    clear_output(wait=True)
    print(f"Loaded {df.shape[0]} rows from {fname}")
    display(df.head())
    break


Please upload your sales CSV now


# 2. Summary and Understanding of Data

## 2.1 Exploratory Data Analysis

In [ ]:
# Function to perform all EDA
def eda(df, name=""):
    # Displaying basic details
    print(f"EDA of {name} Dataset is........")
    print(f"Size: {df.size}")
    print(f"Columns: {df.shape[1]}")
    print(f"Records: {df.shape[0]}")
    print("*"*50, "\n")

    # Displayng Top 4 records of Data
    print("First Look of Data: ")
    display(df.head())
    print("*"*50, "\n")

    # Getting Numerical columns and Categorical columns
    cat_col = df.select_dtypes(object).columns
    num_col = df.select_dtypes(np.number).columns

    # Displaying the Numerical Columns
    print("Dataset has following Numerical Columns: ")
    if len(num_col) == 0:
        print('''\t"No Numerical Column exist."''', "\n")
    else:
        for i, j in enumerate(num_col):
            print(f"{i+1}- {j}")

    # Displaying the Categorical Columns
    print("*"*50)
    print("Dataset has following Categorical Columns: ")
    if len(cat_col) == 0:
        print("\tNo Categorical Column exist.")
    else:
        for i, j in enumerate(cat_col):
            print(f"{i+1}- {j}")
    print("*"*50, "\n")

    # Displaying info of Data e.g., Null values, data types etc
    print("Information of Data is as follows: ")
    display(df.info())
    print("*"*50, "\n")

    # Displaying Statistical properties
    print("Statistical Properties of Data: ")
    display(df.describe(include="all"))
    print("*"*50, "\n")

In [ ]:
eda(df, "Online Retail")

In [ ]:
df2 = df.copy()

# 3. Data Visualization

## 3.1 Product Analysis

### 3.1.1 What are the top-selling products by quantity and revenue?

In [ ]:
# Calculate total quantity and revenue for each product
df2['Revenue'] = df2['Quantity'] * df2['UnitPrice']
df2['Revenue'] = df2['Revenue'].round(2)
product_quantity = df2.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)
product_revenue = df2.groupby('Description')['Revenue'].sum().sort_values(ascending=False).head(10)

In [ ]:
product_quantity

In [ ]:
# Plot top-selling products by quantity
plt.figure(figsize=(10, 4))
sns.barplot(x=product_quantity.values, y=product_quantity.index, palette='viridis')
plt.title('Top Selling Products by Quantity')
plt.xlabel('Quantity Sold')
plt.ylabel('Product Description')
plt.show()

In [ ]:
product_revenue

In [ ]:
# Plot top-selling products by revenue
plt.figure(figsize=(10, 4))
sns.barplot(x=product_revenue.values, y=product_revenue.index, palette='viridis')
plt.title('Top Selling Products by Revenue')
plt.xlabel('Revenue')
plt.ylabel('Product Description')
plt.show()

### 3.1.2 Can you identify any seasonal trends in product sales?

In [ ]:
# Convert 'InvoiceDate' to datetime format if not already
df2['InvoiceDate'] = pd.to_datetime(df2['InvoiceDate'])

# Extract month and year from 'InvoiceDate'
df2['Month'] = df2['InvoiceDate'].dt.month
df2['Year'] = df2['InvoiceDate'].dt.year

# Group by month and year to get total sales
monthly_sales = df2.groupby(['Year', 'Month'])['Quantity'].sum().reset_index()

# Plot the seasonal trends
plt.figure(figsize=(14, 6))
sns.lineplot(x='Month', y='Quantity', hue='Year', data=monthly_sales, palette='viridis')
plt.title('Seasonal Trends in Product Sales')
plt.xlabel('Month')
plt.ylabel('Total Sales Quantity')
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.legend(title='Year')
plt.show()

In [ ]:
monthly_sales

## 3.2 Customer Analysis:

### 3.2.1 Who are the top customers in terms of purchase frequency and total spending?

In [ ]:
# Calculate total spending for each customer
customer_purchase_frequency = df2.groupby('CustomerID')['InvoiceNo'].nunique().sort_values(ascending=False).head(10)
customer_total_spending = df2.groupby('CustomerID')['Revenue'].sum().sort_values(ascending=False).head(10)

In [ ]:
customer_purchase_frequency

In [ ]:
# Plot top customers by purchase frequency
plt.figure(figsize=(12, 6))
sns.barplot(x=customer_purchase_frequency.values, y=customer_purchase_frequency.index, palette='Blues')
plt.title('Top Customers by Purchase Frequency')
plt.xlabel('Number of Purchases')
plt.ylabel('Customer ID')
plt.show()

In [ ]:
customer_total_spending

In [ ]:
# Plot top customers by total spending
plt.figure(figsize=(12, 6))
sns.barplot(x=customer_total_spending.values, y=customer_total_spending.index, palette='Blues')
plt.title('Top Customers by Total Spending')
plt.xlabel('Total Spending')
plt.ylabel('Customer ID')
plt.show()

### 3.2.2 How is the customer distribution across different countries?

In [ ]:
# Plot the customer distribution across different countries with numbers on each bar
plt.figure(figsize=(12, 8))  # Adjust the figure size
sns.countplot(y='Country', data=df2, palette='Set3')  # Use countplot with y-axis for horizontal bars

# Add numbers on each bar
for idx, value in enumerate(df2['Country'].value_counts()):
    plt.text(value + 1, idx, str(value), ha='left', va='center', fontsize=8)

plt.title('Customer Distribution Across Different Countries')
plt.xlabel('Number of Customers')
plt.ylabel('Country')
plt.show()

In [ ]:
# Print the numerical values
for idx, (country, count) in enumerate(df2['Country'].value_counts().items()):
    print(f"{country}: {count} Customers")

### 3.2.3 How many new Customers across different Countries?

In [ ]:
# Print the numerical values
unique_customer_counts = df2.groupby('Country')['CustomerID'].nunique().sort_values(ascending=False)
for country, count in unique_customer_counts.items():
    print(f"{country}: {count} Unique / New Customers")

### 3.2.4 What are Top 10 Countries with New Customers?

In [ ]:
# Select the top 10 countries
top_10_countries = unique_customer_counts.head(10)

# Plot the top 10 countries with unique customers and add numbers on each bar
plt.figure(figsize=(12, 6))
ax = sns.barplot(x=top_10_countries.values, y=top_10_countries.index, palette='Set3')

# Add numbers on each bar
for idx, value in enumerate(top_10_countries.values):
    ax.text(value + 1, idx, str(value), ha='left', va='center', fontsize=8)

plt.title('Top 10 Countries with Unique Customers')
plt.xlabel('Number of Unique Customers')
plt.ylabel('Country')
plt.show()

## 3.3 Country-wise Analysis

### 3.3.1 What is the distribution of transactions across different countries?

In [ ]:
# Plot the distribution of transactions across different countries
plt.figure(figsize=(12, 6))
sns.countplot(x='Country', data=df2, palette='Set3')
plt.title('Distribution of Transactions Across Different Countries')
plt.xlabel('Country')
plt.ylabel('Number of Transactions')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
plt.show()

### 3.3.2 What is the Average transaction cost by country

In [ ]:
# Presentation of the average transaction cost by country
average_transaction_cost_by_country = df.groupby('Country').apply(lambda x: (x['Quantity'] * x['UnitPrice']).mean()).sort_values(ascending=False)
plt.figure(figsize=(12, 6))
sns.barplot(x=average_transaction_cost_by_country.index, y=average_transaction_cost_by_country.values)
plt.title('Average Transaction Cost by Country')
plt.xlabel('Country')
plt.ylabel('Average Transaction Cost')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
average_transaction_cost_by_country

## 3.4 Return and Cancellations

### 3.4.1 How many returns or cancellations are present in the dataset?

In [ ]:
# Count the number of returns or cancellations
returns_cancellations_count = df2[df2['Quantity'] < 0].shape[0]

print(f"Number of Returns or Cancellations: {returns_cancellations_count}")

### 3.4.2 How many unique customers did return or cancellations?

In [ ]:
# Filter for transactions with negative quantities (returns or cancellations)
returns_cancellations = df2[df2['Quantity'] < 0]

# Count the number of unique customers who made returns or cancellations
unique_customers_returns_cancellations = returns_cancellations['CustomerID'].nunique()

print(f"Number of Unique Customers with Returns or Cancellations: {unique_customers_returns_cancellations}")

### 3.4.3 How much revenue lost due to return or cancellations?

In [ ]:
# Calculate total revenue generated
total_revenue_generated = (df2['Quantity'] * df2['UnitPrice']).sum()

# Filter for transactions with negative quantities (returns or cancellations)
returns_cancellations = df2[df2['Quantity'] < 0]

# Calculate total revenue lost due to returns or cancellations
revenue_lost_returns_cancellations = (returns_cancellations['Quantity'] * returns_cancellations['UnitPrice']).sum()

# Take the absolute value to represent revenue lost as positive
revenue_lost_returns_cancellations = abs(revenue_lost_returns_cancellations)

# Calculate net revenue (generated - lost)
net_revenue = total_revenue_generated - revenue_lost_returns_cancellations

# Calculate percentages
percentage_revenue_lost = (revenue_lost_returns_cancellations / total_revenue_generated) * 100
percentage_net_revenue = (net_revenue / total_revenue_generated) * 100

print(f"Total Revenue Generated: ${total_revenue_generated:.2f}")
print(f"Total Revenue Lost due to Returns or Cancellations: ${revenue_lost_returns_cancellations:.2f} ({percentage_revenue_lost:.2f}%)")
print(f"Net Revenue (Generated - Lost): ${net_revenue:.2f} ({percentage_net_revenue:.2f}%)")

### 3.4.4 Which month and year has the maximum number of return or cancellations and how many return or cancellations in that particular month and year?

In [ ]:
# First, filter for returns/cancellations (negative Quantity)
returns = df[df['Quantity'] < 0]

# Group by Year & Month and count
returns_by_month_year = (
    returns
      .groupby([
          returns['InvoiceDate'].dt.year.rename('Year'),
          returns['InvoiceDate'].dt.month.rename('Month')
      ])
      .size()
      .reset_index(name='Count')
)

# Guard: only compute idxmax if we have any data
if returns_by_month_year.empty:
    print("No returns or cancellations in this dataset.")
else:
    idx = returns_by_month_year['Count'].idxmax()
    max_row     = returns_by_month_year.loc[idx]
    max_year    = max_row['Year']
    max_month   = max_row['Month']
    max_count   = max_row['Count']
    print("Month and Year with Maximum Returns or Cancellations:")
    print(f" • {max_year}-{max_month}: {max_count} returns/cancellations")


### 3.4.5 Which products has the highest number of return or cancellations?

In [ ]:
# Filter for returns/cancellations (negative Quantity)
returns = df[df['Quantity'] < 0]

# Group by product (Description) and count
returns_by_product = (
    returns
      .groupby('Description')
      .size()
      .reset_index(name='Count')
)

# Guard against empty result
if returns_by_product.empty:
    print("No returns or cancellations in this dataset, so no product to report.")
else:
    idx = returns_by_product['Count'].idxmax()
    max_row = returns_by_product.loc[idx]
    max_product_description = max_row['Description']
    max_product_count       = max_row['Count']

    print("Product with Maximum Returns or Cancellations:")
    print(f" • {max_product_description}: {max_product_count} cases")


### 3.4.6 Top 10 Products with highest number of return or cancellations?

In [ ]:
# Filter for returns/cancellations
returns = df[df['Quantity'] < 0]

# Group by product and count
returns_by_product = (
    returns
      .groupby('Description')
      .size()
      .reset_index(name='Count')
)

# Guard against empty result
if returns_by_product.empty:
    print("No returns or cancellations in this dataset, so no products to list.")
else:
    # Sort descending and take top 10
    top_10 = returns_by_product.sort_values(by='Count', ascending=False).head(10)
    print("Top 10 Products with highest returns or cancellations:")
    display(top_10[['Description','Count']])


# 4. Data Preparation

## 4.1 Handling Null Values

In [ ]:
# 4.1 Handling Null Values

# 1) Report
print("Null counts before:")
display(df.isna().sum())

# 2) Act
df['Description'].fillna('Missing', inplace=True)             # keep the row, mark missing
df = df.dropna(subset=['InvoiceDate', 'UnitPrice'])           # drop rows missing these critical fields
df['CustomerID'].fillna(0, inplace=True)                      # impute zeros for anonymous customers

# 3) Verify
print("\nNull counts after:")
display(df.isna().sum())
print(f"Data shape after dropping critical nulls: {df.shape}")


## 4.2 Handling Duplicate & Outliers

In [ ]:
# 1) Drop full-row duplicates
before = df.shape[0]
df = df.drop_duplicates()
dropped = before - df.shape[0]
print(f"Dropped {dropped} full-row duplicates")

# 2) Compute TotalSales (needed for outlier detection)
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

# 3) Identify and drop outliers (TotalSales > mean + 3·std)
mean, std = df['TotalSales'].mean(), df['TotalSales'].std()
cutoff = mean + 3 * std
outliers = df[df['TotalSales'] > cutoff]
print(f"Identified {len(outliers)} outliers (TotalSales > {cutoff:.2f})")
df = df[df['TotalSales'] <= cutoff]
print(f"Data shape after outlier removal: {df.shape}")


## 4.3 Handling Returns & Cancellations
Separate any negative-Quantity rows (returns) for reporting, and keep only non-returns for the main dataset.


In [ ]:
df_all      = df.copy()                     # keep full set including returns
df_returns  = df_all[df_all['Quantity'] < 0] # extract returns
df          = df_all[df_all['Quantity'] >= 0]# main dataset without returns

print(f"Extracted {len(df_returns)} returns")
print(f"Main dataset now has {df.shape[0]} sales records")


## 4.4 Feature Engineering: Date Parts
Create Year, Month, and Day-of-Week features from the InvoiceDate for analysis and modelling.


In [ ]:
df['Year']      = df['InvoiceDate'].dt.year
df['Month']     = df['InvoiceDate'].dt.month
df['DayOfWeek'] = df['InvoiceDate'].dt.day_name()

print("Added Year, Month, and DayOfWeek columns")
display(df[['InvoiceDate','Year','Month','DayOfWeek']].head(5))


## 5. Feature Engineering

# 5.1 Selecting relevant Columns

In [ ]:
# Select relevant columns
selected_columns = ['Quantity', 'InvoiceDate', 'UnitPrice', 'Country', 'TotalSales']

# Create a new DataFrame with selected columns
df_selected = df[selected_columns].copy()

# Display the new DataFrame
display("Selected Columns for Regression Analysis:")
display(df_selected.head())

# 5.2 One-Hot Encoding

In [ ]:
# Perform one-hot encoding for the 'Country' column
df_selected = pd.get_dummies(df_selected, columns=['Country'], prefix='Country')

In [ ]:
df_selected.head()

In [ ]:
# Extract features from 'InvoiceDate'
df_selected['InvoiceDate'] = pd.to_datetime(df_selected['InvoiceDate'])
df_selected['DayOfWeek'] = df_selected['InvoiceDate'].dt.dayofweek
df_selected['Hour'] = df_selected['InvoiceDate'].dt.hour
df_selected['Month'] = df_selected['InvoiceDate'].dt.month

# Drop the original 'InvoiceDate' column
df_selected = df_selected.drop('InvoiceDate', axis=1)

## 5.3 Separating Target Variable

In [ ]:
# Separate features and target variable
X = df_selected.drop('TotalSales', axis=1)
y = df_selected['TotalSales']

## 5.4 Train-test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Standardization for Linear Regression**

In [ ]:
# Standardization for Linear Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[['Quantity', 'UnitPrice']])
X_test_scaled = scaler.transform(X_test[['Quantity', 'UnitPrice']])

# Regression Problem Statement

Can we predict the total sales amount (in dollars) for a given transaction based on the quantity of items purchased, the unit price of each item, the country of the customer, and the timestamp of the transaction?

# 6. Regressor Models Implementation

In [ ]:
# check the performance on diffrent regressor
models = []
models.append(('LinearRegression', LinearRegression()))
models.append(('Random Forest', RandomForestRegressor()))
models.append(('Decision Tree', DecisionTreeRegressor()))
models.append(('Ridge', Ridge()))
models.append(('KNeighborsRegressor', KNeighborsRegressor()))

train_l = []
test_l = []
mae_l = []
rmse_l = []
r2_l = []

import time
i = 0
for name,model in models:
    i = i+1
    start_time = time.time()

    # Fitting model to the Training set
    clf = model
    clf.fit(X_train, y_train)

    # Scores of model
    train = model.score(X_train, y_train)
    test = model.score(X_test, y_test)

    train_l.append(train)
    test_l.append(test)

    # predict values
    predictions = clf.predict(X_test)
    # RMSE
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    rmse_l.append(rmse)
    # MAE
    mae = mean_absolute_error(y_test,predictions)
    mae_l.append(mae)
    # R2 score
    r2 = r2_score(y_test,predictions)
    r2_l.append(r2)



    print("+","="*100,"+")
    print('\033[1m' + f"\t\t\t{i}-For {name} The Performance result is: " + '\033[0m')
    print("+","="*100,"+")
    print('Training Score  : ', train)
    print("-"*50)
    print('Testing Score   : ', test)
    print("-"*50)
    print('Root mean squared error (RMSE) : ', rmse)
    print("-"*50)
    print('Mean absolute error (MAE) : ', mae)
    print("-"*50)
    print('R2 score : ', r2)
    print("-"*50)



    print("\t\t\t\t\t\t\t-----------------------------------------------------------")
    print(f"\t\t\t\t\t\t\t Time for detection ({name}) : {round((time.time() - start_time), 3)} seconds...")
    print("\t\t\t\t\t\t\t-----------------------------------------------------------")
    print()

In [ ]:
comp = pd.DataFrame({"Models": dict(models).keys(), "Training Score": train_l, "Testing Score": test_l, "MAE": mae_l, "RMSE": rmse_l, "R2 Score": r2_l})
comp


In [ ]:
plt.figure(figsize=(14, 8))

# Plotting Training Score
plt.subplot(2, 3, 1)
sns.barplot(x='Models', y='Training Score', data=comp)
plt.title('Training Score')
plt.xticks(rotation=45, ha='center')

# Plotting Testing Score
plt.subplot(2, 3, 2)
sns.barplot(x='Models', y='Testing Score', data=comp)
plt.title('Testing Score')
plt.xticks(rotation=45, ha='center')

# Plotting MAE
plt.subplot(2, 3, 3)
sns.barplot(x='Models', y='MAE', data=comp)
plt.title('Mean Absolute Error (MAE)')
plt.xticks(rotation=45, ha='center')

# Plotting RMSE
plt.subplot(2, 3, 4)
sns.barplot(x='Models', y='RMSE', data=comp)
plt.title('Root Mean Squared Error (RMSE)')
plt.xticks(rotation=45, ha='center')

# Plotting R2 Score
plt.subplot(2, 3, 5)
sns.barplot(x='Models', y='R2 Score', data=comp)
plt.title('R2 Score')
plt.xticks(rotation=45, ha='center')

plt.tight_layout()
plt.show()